In [1]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [2]:
PINECONE_API_KEY = "c126881a-2588-49b0-b24d-f1f3bae35622"
PINECONE_API_ENV = "gcp-starter"

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
'''text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=500,
                                    chunk_overlap=20)'''
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
#text_chunks=text_splitter.split_documents(extracted_data)
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [7]:
len(text_chunks)

7020

In [8]:
text_chunks[0]

Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION', metadata={'source': 'data\\Medical_book.pdf', 'page': 1})

In [9]:
'''embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')'''
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

d:\ana\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

384

In [14]:
query_result

[-0.034477267414331436,
 0.031023206189274788,
 0.006734929047524929,
 0.026108982041478157,
 -0.03936203196644783,
 -0.16030243039131165,
 0.06692398339509964,
 -0.006441489793360233,
 -0.04745049402117729,
 0.014758839271962643,
 0.07087527960538864,
 0.05552763119339943,
 0.01919335499405861,
 -0.026251323521137238,
 -0.010109569877386093,
 -0.02694045566022396,
 0.022307392209768295,
 -0.02222662791609764,
 -0.14969263970851898,
 -0.017493031919002533,
 0.007676294539123774,
 0.054352253675460815,
 0.0032544503919780254,
 0.031725917011499405,
 -0.08462144434452057,
 -0.029405998066067696,
 0.051595620810985565,
 0.048124048858881,
 -0.00331486901268363,
 -0.058279212564229965,
 0.041969265788793564,
 0.022210726514458656,
 0.12818878889083862,
 -0.02233896590769291,
 -0.011656217277050018,
 0.06292834132909775,
 -0.032876402139663696,
 -0.09122605621814728,
 -0.03117532841861248,
 0.05269954353570938,
 0.047034818679094315,
 -0.08420311659574509,
 -0.030056139454245567,
 -0.020744

In [15]:
'''import os

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'a27927a1-3870-4848-b5c5-db3c656fc02f')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')'''

"import os\n\nPINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'a27927a1-3870-4848-b5c5-db3c656fc02f')\nPINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')"

In [16]:
import pinecone
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "medical-chatbot" # put in the name of your pinecone index here


In [17]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [18]:
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [18]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [20]:
llm=CTransformers(model="D:\medical_chatbot\model\llama-2-7b-chat.ggmlv3.q4_0_2.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [21]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [22]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

d:\ana\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  Acne is a skin disease characterized by pimples on the face, chest, and back that occur when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


PineconeProtocolError: Failed to connect; did you specify the correct index name?